In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [ ]:
# Global Init Variables
GET_WEATHER = False

In [ ]:
# read observations file that already has the PowderyMildew feature
observations_df = pd.read_csv('observations.csv')

# Drop rows with missing values
observations_df = observations_df.dropna()

observations_df.head()

In [ ]:
# Function to parse date string and extract components
def extract_date_components(date_string):
    import datetime
    # Parse ISO format date string
    dt = datetime.datetime.fromisoformat(date_string.replace('Z', '+00:00'))
    return dt.year, dt.month, dt.day

# Extract relevant data for true observations
lons = []
lats = []
years = []
months = []
days = []

for _, row in observations_df.iterrows():
    # Parse the coordinates string into a list of floats
    coords = eval(row['coordinates'])
    lons.append(coords[0])
    lats.append(coords[1])
    
    # Extract the date components from the date_string
    year, month, day = extract_date_components(row['date_string'])
    years.append(year)
    months.append(month)
    days.append(day)
    
# Apply parsed data to the observations_df
observations_df['longitude'] = lons
observations_df['latitude'] = lats
observations_df['year'] = years
observations_df['month'] = months
observations_df['day'] = days

observations_df.head()


In [ ]:
# Get NASA weather data at given coordinates 7 days before and after the given date
# 
# Parameters:
#   lon (float): Longitude coordinate
#   lat (float): Latitude coordinate
#   day (int): Day of the month
#   month (int): Month of the year
#   year (int): Year
#   param_list (list): List of weather parameters to retrieve
#
# Returns:
#   dict: Dictionary containing the requested weather parameters with their values,
#

def get_general_weather(lon, lat, day, month, year, param_list):
    url = "https://power.larc.nasa.gov/api/temporal/hourly/point"
    
    month_str = str(month).zfill(2)  # make sure is 2 length
    day_str = str(day).zfill(2)      # same as above
    start_date = f"{year}-{month_str}-{day_str}"

    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d") 
    end_date = start_date + datetime.timedelta(days=7)
    start_date -= datetime.timedelta(days=7)

    start_date = start_date.strftime("%Y%m%d")
    end_date = end_date.strftime("%Y%m%d")
    
    parameters = {
        "parameters": ",".join(param_list),
        "community": "AG",
        "latitude": lat,
        "longitude": lon,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }
    curr_data = {}

    response = requests.get(url, params=parameters)
    data = response.json()
    try:
        for param in param_list:
            curr_data[param] = data['properties']['parameter'][param]
    except:
        print(f"Error getting data for {year}-{month_str}-{day_str} at {lat},{lon}")
        return None
    
    return curr_data

In [ ]:
# If GET_WEATHER is True, fetch weather data for each observation, save to weather_data.csv

if GET_WEATHER:
    """
    T2M	Temperature at 2 Meters
    RH2M	Relative Humidity at 2 Meters
    WS2M	Wind Speed at 2 Meters
    T2MDEW	Dew/Frost Point at 2 Meters
    T2MWET	Wet Bulb Temperature at 2 Meters
    QV2M	Specific Humidity at 2 Meters
    """
    data = []
    dataframe = pd.DataFrame(columns=['Datetime','id', 'Temperature', 'Humidity', 'Wind Speed', 'Dew/Frost Point', 'Wet Bulb Temperature', 'Specific Humidity'])

    for i, row in observations_df.iterrows():
        weather_data = get_general_weather(row['longitude'], row['latitude'], row['day'], row['month'], row['year'], ['T2M', 'RH2M', 'WS2M', 'T2MDEW', 'T2MWET', 'QV2M'])
        if weather_data is None:
            for j in range(3):
                weather_data = get_general_weather(row['longitude'], row['latitude'], row['day'], row['month'], row['year'], ['T2M', 'RH2M', 'WS2M', 'T2MDEW', 'T2MWET', 'QV2M'])
                if weather_data is not None:
                    break
        if weather_data is None:    
            continue
        dates = list(weather_data['T2M'].keys())
        id_vals = [row['id']] * len(dates)
        temp_values = list(weather_data['T2M'].values())
        humidity_values = list(weather_data['RH2M'].values())
        wind_speed_values = list(weather_data['WS2M'].values())
        dew_frost_values = list(weather_data['T2MDEW'].values())
        wet_bulb_values = list(weather_data['T2MWET'].values())
        specific_humidity_values = list(weather_data['QV2M'].values())
        
        curr_df = pd.DataFrame({
            'Datetime': dates,
            'id': id_vals,
            'Temperature': temp_values,
            'Humidity': humidity_values,
            'Wind Speed': wind_speed_values,
            'Dew/Frost Point': dew_frost_values,
            'Wet Bulb Temperature': wet_bulb_values,
            'Specific Humidity': specific_humidity_values
        })
        
        dataframe = pd.concat([dataframe, curr_df], ignore_index=True)
        # Print progress every 10 rows
        if i % 10 == 0:
            print(f"Processed {i}")
        
    dataframe.to_csv('weather_data.csv', index=False)

In [ ]:
observations_df.head()

In [ ]:
# Load the weather data if not already loaded
from datetime import datetime, timedelta

# Load the weather data
weather_df = pd.read_csv('weather_data.csv')

# convert datetime to same format as observations_df
weather_df['Datetime'] = pd.to_datetime(weather_df['Datetime'], format='%Y%m%d%H')
weather_df['Date'] = weather_df['Datetime'].dt.date

# Use same format as weather_df
observations_df['Date'] = pd.to_datetime(observations_df[['year', 'month', 'day']])

# Group by id and Date to find all unique days per ID
day_indices = weather_df.groupby(['id', 'Date']).first().reset_index()

# make dict of observation ID with original date
observation_dates = {}
for _, row in observations_df.iterrows():
    observation_dates[row['id']] = row['Date']

# Calculate difference between observation date and weather date
def get_relative_day(row):
    obs_date = observation_dates.get(row['id'])
    this_date = pd.to_datetime(row['Date'])
    difference = (this_date - obs_date).days
    return difference

# Apply the function to get relative day
day_indices['RelativeDay'] = day_indices.apply(get_relative_day, axis=1)

# Add relative day to weather_df
day_map = day_indices.set_index(['id', 'Date'])['RelativeDay']
weather_df['RelativeDay'] = weather_df.set_index(['id', 'Date']).index.map(day_map)

# Variables to calculate aggregations for
weather_vars = ['Temperature', 'Humidity', 'Wind Speed', 'Dew/Frost Point', 
                'Wet Bulb Temperature', 'Specific Humidity']

# Time windows to aggregate over (days before and after the observation day)
time_windows = {
    'day_of': (0, 0),
    'day_before_after': (-1, 1),
    'three_days': (-3, 3),
    'one_week': (-7, 7),
    'two_weeks': (-14, 14)
}

# Function to calculate aggregates given ID and time window
def calculate_aggregates(weather_subset, var_name, window_name, start_day, end_day):
    # Filter data for given time window
    window_data = weather_subset[(weather_subset['RelativeDay'] >= start_day) & (weather_subset['RelativeDay'] <= end_day)]
    
    # Calculate aggregate and return as a series
    result = pd.Series({
        f"{var_name}_{window_name}_mean": window_data[var_name].mean(),
        f"{var_name}_{window_name}_min": window_data[var_name].min(),
        f"{var_name}_{window_name}_max": window_data[var_name].max(),
        f"{var_name}_{window_name}_median": window_data[var_name].median()
    })
    
    return result

# Create empty DataFrame to store results
all_features = []
result_rows = []

# For each observation row, calculate all aggregate weather data
for id_val in observations_df['id'].unique():
    id_weather = weather_df[weather_df['id'] == id_val]
        
    # Dictionary to store aggregated values for this ID
    id_features = {'id': id_val}
    
    # Iterate over all weather data variables
    for var in weather_vars:
        # iterate over all items in time_windows
        for window_name, (start_day, end_day) in time_windows.items():
            aggs = calculate_aggregates(id_weather, var, window_name, start_day, end_day)
            id_features.update(aggs)
            
            # Add feature names to our list (only once)
            if id_val == observations_df['id'].unique()[0]:
                all_features.extend(aggs.index.tolist())
    
    result_rows.append(id_features)

# Create DataFrame from results
aggregated_features = pd.DataFrame(result_rows)

# Merge with the original observations dataframe
observations_df = observations_df.merge(aggregated_features, on='id', how='left')

# Create list of feature names
weather_feature_names = all_features

print(f"Created {len(weather_feature_names)}  features")

In order to decrease relationships between variables (such as the connections and potential spurrious correlations between powdery mildew and date_string, month, and day) we can combine this date information into one number: the day of the calendar year 1-365.

In [ ]:
# Calculate day of year from year, month, and day
from datetime import datetime

# Add day of year as a feature to observations_df
observations_df['day_of_year'] = observations_df.apply(
    lambda row: datetime(row['year'], row['month'], row['day']).timetuple().tm_yday, 
    axis=1
)

# Drop rows with NaN values
observations_df = observations_df.dropna()

# Display the updated DataFrame
observations_df.head()


Now we are ready to build a regression tree using this updated data frame!

In [ ]:
# Define the features and target variable
features = weather_feature_names + ['day_of_year']
target = 'PowderyMildew'

X = observations_df[features]
y = observations_df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

#Save the model to be used in the api
joblib.dump(rf_model, 'model.randomforest')

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['False', 'True'], yticklabels=['False', 'True'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



Sensitivity has improved to 72%

In [ ]:
# Feature Importance
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Select top 20 features
top_20_features = feature_importance_df.head(20)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=top_20_features)
plt.title('Top 20 Feature Importances')
plt.show()



Temperature and seasonality are very important.